<table>
 <tr align=left><td><img align=left src="./images/CC-BY.png">
 <td>Text provided under a Creative Commons Attribution license, CC-BY. All code is made available under the FSF-approved MIT license. (c) Kyle T. Mandli</td>
</table>

In [1]:
from __future__ import print_function

%matplotlib inline

import os
import numpy
import matplotlib.pyplot as plt
import matplotlib.animation

from IPython.display import HTML
from clawpack import pyclaw
from clawpack import riemann

# Nonlinear Systems of Conservation Laws

We now turn to problems that are nonlinear and a system of equations.  This theory will combine ideas from both linear systems and nonlinear scalar equations.  In general this theory will split a single discontinuity into $m$ waves where $m$ is the number of equations.

This theory will presented primarily in the context of the 1-dimensional shallow water equations so that we can look at a concrete example.

## Shallow Water

The shallow water equations can most directly be derived from the invicid Navier-Stokes equations defined by
$$\begin{aligned}
    &u_x + w_z = 0 & & \text{incompressibility} \\
    &u_t + (u^2)_x + (uw)_z = -\frac{1}{\rho} P_x & & \text{x-momentum} \\
    &u_t + (u^2)_x + (uw)_z = -\frac{1}{\rho} P_z - g & & \text{z-momentum}
\end{aligned}$$
along with the boundary conditions
$$
    \left \{ \begin{aligned}
        &w = \eta_t + u \eta_x \\
        &P = P_a
    \end{aligned} \right . \quad \quad z = \eta
$$
and
$$
    w = ub_x \quad z = b
$$

### Non-Dimensionalization

Since the shallow water equations are based on a scaling with respect to "shallowness" we will descibe that exansion but reintroduce a dimsional version in closing.

To non-dimensionalize these equations we use the following scalings:
$$
    \frac{x}{\lambda} = \widetilde{x} \quad \quad \frac{z}{a} = \widetilde{z} \quad \quad \frac{t}{T} = \widetilde{t}\\
    \frac{u}{U} = \widetilde{u} \quad \quad \frac{w}{W} = \widetilde{w} \quad \quad \frac{P}{P_0} = \widetilde{P}
$$
Using these scalings we can then define
$$
    \epsilon = \frac{a}{\lambda} \quad \quad U = \sqrt{ga} \quad \quad W = \epsilon U \quad \quad T = \frac{\lambda}{U}
$$
where $\epsilon$ the shallow water parameter relating wave-length to depth and $g$ is gravitational acceleration.

$$
    \frac{x}{\lambda} = \widetilde{x} \quad \quad \frac{z}{a} = \widetilde{z} \quad \quad \frac{t}{T} = \widetilde{t}\\
    \frac{u}{U} = \widetilde{u} \quad \quad \frac{w}{W} = \widetilde{w} \quad \quad \frac{P}{P_0} = \widetilde{P} \\
    \epsilon = \frac{a}{\lambda} \quad \quad U = \sqrt{ga} \quad \quad W = \epsilon U \quad \quad T = \frac{\lambda}{U}
$$
Applying the scalings above we can transform the incompressibility equation to
$$
\begin{aligned}
    &u_x + w_z = 0 \Rightarrow \\
    &\frac{U}{\lambda} \widetilde{u}_{\widetilde{x}} + \frac{W}{a} \widetilde{w}_{\widetilde{z}} = 0 \Rightarrow \\
    &\widetilde{u}_{\widetilde{x}} + \frac{\epsilon U \lambda}{a U} \widetilde{w}_{\widetilde{z}} = 0 \Rightarrow \\
    &\widetilde{u}_{\widetilde{x}} + \widetilde{w}_{\widetilde{z}} = 0.
\end{aligned}
$$

$$
    \frac{x}{\lambda} = \widetilde{x} \quad \quad \frac{z}{a} = \widetilde{z} \quad \quad \frac{t}{T} = \widetilde{t}\\
    \frac{u}{U} = \widetilde{u} \quad \quad \frac{w}{W} = \widetilde{w} \quad \quad \frac{P}{P_0} = \widetilde{P} \\
    \epsilon = \frac{a}{\lambda} \quad \quad U = \sqrt{ga} \quad \quad W = \epsilon U \quad \quad T = \frac{\lambda}{U}
$$
Applying the scalings again we can transform the x-momentum equation to
$$\begin{aligned}
    &u_t + (u^2)_x + (uw)_z = -\frac{1}{\rho} P_x \Rightarrow \\
    &\frac{U}{T} \widetilde{u}_{\widetilde{t}} + \frac{U^2}{\lambda} (\widetilde{u}^2)_{\widetilde{x}} + \frac{UW}{a} (\widetilde{u}\widetilde{w})_{\widetilde{z}} = -\frac{P_0}{\rho \lambda} \widetilde{P}_{\widetilde{x}} \Rightarrow \\
    &\tilde{u}_{\widetilde{t}} + \frac{UT}{\lambda} (\widetilde{u}^2)_{\widetilde{x}} + \frac{TW}{a} (\widetilde{u}\widetilde{w})_{\widetilde{z}} = -\frac{TP_0}{\rho \lambda U} \widetilde{P}_{\widetilde{x}} \Rightarrow \\
    &\tilde{u}_{\tilde{t}} + (\tilde{u}^2)_{\tilde{x}} + (\widetilde{u}\widetilde{w})_{\widetilde{z}} = -\frac{P_0}{\rho U^2} \widetilde{P}_{\widetilde{x}} 
\end{aligned}$$
If we now use the scaling $\frac{P_0}{\rho U^2} = 1$ then we arrive at the appropriate non-dimensional x-momentum PDE:
$$
    \widetilde{u}_{\widetilde{t}} + (\widetilde{u}^2)_{\widetilde{x}} + (\widetilde{u}\widetilde{w})_{\widetilde{z}} = -\widetilde{P}_{\widetilde{x}} 
$$

$$
    \frac{x}{\lambda} = \widetilde{x} \quad \quad \frac{z}{a} = \widetilde{z} \quad \quad \frac{t}{T} = \widetilde{t}\\
    \frac{u}{U} = \widetilde{u} \quad \quad \frac{w}{W} = \widetilde{w} \quad \quad \frac{P}{P_0} = \widetilde{P} \\
    \epsilon = \frac{a}{\lambda} \quad \quad U = \sqrt{ga} \quad \quad W = \epsilon U \quad \quad T = \frac{\lambda}{U}
$$
Finally applying the same procedure to the z-momentum equations we have
$$\begin{aligned}
    &w_t + (uw)_x + (w^2)_z = -\frac{1}{\rho} P_z - g \Rightarrow \\
    &\frac{W}{T}\widetilde{w}_{\widetilde{t}} + \frac{UW}{\lambda}(\widetilde{u}\widetilde{w})_{\widetilde{x}} + \frac{W^2}{a}(\widetilde{w}^2)_{\widetilde{z}} = -\frac{P_0}{\rho a} \widetilde{P}_{\widetilde{z}} - g \Rightarrow \\
    &\widetilde{w}_{\widetilde{t}} + \frac{UT}{\lambda}(\widetilde{u}\widetilde{w})_{\widetilde{x}} + \frac{WT}{a}(\widetilde{w}^2)_{\widetilde{z}} = -\frac{T}{W} \left (\frac{P_0}{\rho a} \widetilde{P}_{\tilde{z}} + g \right ) \Rightarrow \\
    &\widetilde{w}_{\widetilde{t}} + (\widetilde{u}\widetilde{w})_{\widetilde{x}} + (\widetilde{w}^2)_{\widetilde{z}} = -\frac{\lambda}{\epsilon U^2} \left (\frac{P_0}{\rho a} \widetilde{P}_{\widetilde{z}} + g \right ) \Rightarrow \\
    &\widetilde{w}_{\widetilde{t}} + (\widetilde{u}\widetilde{w})_{\widetilde{x}} + (\widetilde{w}^2)_{\widetilde{z}} = -\frac{1}{g \epsilon^2} \left (g \widetilde{P}_{\widetilde{z}} + g \right ) \Rightarrow \\
    &\epsilon^2 \left (\widetilde{w}_{\widetilde{t}} + (\widetilde{u}\widetilde{w})_{\widetilde{x}} + (\widetilde{w}^2)_{\widetilde{z}} \right )= - \widetilde{P}_{\widetilde{z}} - g
\end{aligned}
$$

For the boundary conditions we also need to introduce these scalings along with $\frac{\eta}{\delta} = \widetilde{\eta}$ and $\frac{b}{\beta} = \widetilde{b}$:
$$\begin{aligned}
    &w = \eta_t + u \eta_x \Rightarrow \\
    &\epsilon U \widetilde{w} = \frac{\delta U}{\lambda} \widetilde{\eta}_{\widetilde{t}} + \frac{U\delta}{\lambda} \widetilde{u} \widetilde{\eta}_{\widetilde{x}} \Rightarrow \\
    &\epsilon \frac{\lambda}{\delta} \widetilde{w} = \widetilde{\eta}_{\widetilde{t}} + \widetilde{u} \widetilde{\eta}_{\widetilde{x}}
\end{aligned}$$
and
$$\begin{aligned}
    &w = u b_x \Rightarrow \\
    &\epsilon \frac{\lambda}{\beta} \widetilde{w} = \widetilde{u} \widetilde{b}_{\widetilde{x}}
\end{aligned}$$

For the upper boundary condition, if $\frac{\delta}{\lambda} = \mathcal{O}(\epsilon)$ the boundary condition is all of the same order.  This is probably not the case as $\delta \ll a$ unless in very shallow water.

The lower boundary condition is a similar situation, if $\frac{\beta}{\lambda} = \mathcal{O}(\epsilon)$ the boundary condition is all of the same order.  In the case of the bathymetry scale, this is probably the case as the bathymetry will vary on the scale of the depth scale $a$.

The final non-dimensionalized equations are then (dropping the $\widetilde{\cdot}$):
$$\begin{aligned}
    &u_x + w_z = 0 \\
    &u_t + (u^2)_x + (uw)_z = -P_x \\
    &\epsilon^2 \left (w_t + (uw)_x + (w^2)_z \right ) = -P_z - 1
\end{aligned}$$
with boundary conditions:
$$\begin{aligned}
        &\epsilon \frac{\lambda}{\delta}w = \eta_t + u \eta_x \quad P = P'_a \quad &z = \eta\\
        &\epsilon \frac{\lambda}{\beta}w = u b_x &z = b
\end{aligned}$$
where $P'_a$ is the appropriately scaled atmospheric pressure condition.

### Depth Integration

Integrating the non-dimensionalized incompressibility equations gives us:
$$\begin{aligned}
        &\int^{\eta}_b \left ( u_x + w_z \right) dz = 0 \Rightarrow \\
        &\frac{\partial}{\partial x} \int^{\eta}_b u dz - u \eta_x |_{z=\eta} + u b_x |_{z=b} + w|_{z=\eta} - w|_{z=b} = 0 \Rightarrow \\
        &\frac{\partial}{\partial x} \int^{\eta}_b u dz + \left [w - u \eta_x\right ]_{z=\eta} + \left [u b_x - w \right]_{z=b} = 0 \Rightarrow \\
        &\frac{\partial}{\partial x} \int^{\eta}_b u dz + \eta_t = 0 \Rightarrow \\
        &h_t + \frac{\partial}{\partial x} \int^{\eta}_b u dz = 0
\end{aligned}$$

Before we integrate the momentum equations we will introduce one of the two assumptions made to simplify the original equations.
1. The **hydrostatic assumption** assumes that the pressure is dominated by the gravitational force:
$$
    P = P_a + (\eta - z).
$$
1. The **differential advection** assumption assumes that there is little variation of the horizontal velocity $u$ through the water vertically.  This then implies that
$$
    \overline{f \cdot g} = \overline{f} \cdot \overline{g}
$$
where
$$
    \overline{(f \cdot g)} = \frac{1}{h} \int^\eta_b f \cdot g dz.
$$

Moving back to the integration of the momentum equations we will first start with the z-momentum equations, since the z-momentum equation is multiplied by what we will assume is a small parameter, i.e. $\epsilon \ll 1$, we can simplify the vertical momentum equation to
$$\begin{aligned}
    0 &= - P_z - 1 \\
    P_z &= -1 \\
    P &= P_a + (\eta - z),
\end{aligned}$$
which now justifies our assumption of hydrostatic equilibrium.

For the x-momentum equation we have now
$$\begin{aligned}
    &\int^{\eta}_b \left ( u_t + (u^2)_x + (uw)_z \right) dz = - \int^{\eta}_b (P_a + (\eta - z))_x dz \Rightarrow \\
    &\frac{\partial}{\partial t} \int^{\eta}_b u dz - \eta_t u|_{z=\eta} + \frac{\partial}{\partial x} \int^{\eta}_b u^2 dz - \eta_x u^2|_{z=\eta} + b_x u^2|_{z=b} + uw|_{z=\eta} - uw|_{z=b} = - \int^{\eta}_b (\eta)_x dz \\
    &\frac{\partial}{\partial t} \int^{\eta}_b u dz + \frac{\partial}{\partial x} \int^{\eta}_b u^2 dz  - \left [u (\eta_t + \eta_x u - w) \right]_{z=\eta} + \left[u (b_x u - w)\right]_{z=b} = - \frac{\partial}{\partial x} \int^{\eta}_b \eta dz + \eta_x \eta - b_x \eta \\
    &\frac{\partial}{\partial t} \int^{\eta}_b u dz + \frac{\partial}{\partial x} \int^{\eta}_b u^2 dz = \frac{\partial}{\partial x} \left[ -\eta h + \frac{1}{2} \eta^2 \right ] - b_x \eta \\
        &\frac{\partial}{\partial t} \int^{\eta}_b u dz + \frac{\partial}{\partial x} \int^{\eta}_b u^2 dz = \frac{\partial}{\partial x} \left[ -h^2 + hb  + \frac{1}{2} (h^2 + 2hb + b^2) \right ] - b_x (h+b) \Rightarrow \\
        &\frac{\partial}{\partial t} \int^{\eta}_b u dz + \frac{\partial}{\partial x} \int^{\eta}_b u^2 dz = \frac{\partial}{\partial x} \left[ - \frac{1}{2} h^2 \right ] - b_x h\\
        &\frac{\partial}{\partial t} \int^{\eta}_b u dz + \frac{\partial}{\partial x} \left ( \int^{\eta}_b u^2 dz + \frac{1}{2} h^2 \right ) = - b_x h
\end{aligned}$$

Now if we use our differential advection assumption and simplify the notation
$$
    \overline{u} = \frac{1}{h} \int^{\eta}_b u dz \quad \quad
    \overline{u^2} = \frac{1}{h} \int^{\eta}_b u^2 dz
$$
we then have from the incompressibility integration
$$\begin{aligned}
    &h_t + \frac{\partial}{\partial x} \int^{\eta}_b u dz = 0 \\
    &h_t + (h \overline{u})_x = 0
\end{aligned}$$
and the x-momentum equation
$$\begin{aligned}
    &\frac{\partial}{\partial t} \int^{\eta}_b u dz + \frac{\partial}{\partial x} \left ( \int^{\eta}_b u^2 dz + \frac{1}{2} h^2 \right ) = - b_x h \\
    & (h\overline{u})_t + \left ( h \overline{u}^2 + \frac{1}{2} h^2 \right )_x = - b_x h.
\end{aligned}$$
Finally dropping the averaging bars we are lead to the shallow water equations:
$$\begin{aligned}
    &h_t + (hu)_x = 0 \\
    &(hu)_t + \left (hu^2 + \frac{1}{2} h^2 \right)_x = -h b_x
\end{aligned}$$

Although these above equations are a valid we will simplify our discussion and drop the term due to bathymetry $b$ and also reapply the dimensional version leading to our final form of the equations
$$\begin{aligned}
    &h_t + (hu)_x = 0 \\
    &(hu)_t + \left (hu^2 + \frac{1}{2} g h^2 \right)_x = 0
\end{aligned}$$
leading to a conservation law.

Note that we can also derive this set of equations from our more direct methods of considering fluxes but some assumptions must be made that makes these equations more difficult to justify.  See the textbook if you would like to this derivation.

Analyzing this equation in the context of systems of hyperbolic eqautions the first order of business is to compute the Jacobian of the flux function.  First define the vector of the solution as
$$
    q(x,t) = \begin{bmatrix} h \\ hu \end{bmatrix}
$$
and the flux function as
$$
    f(q) = \begin{bmatrix} hu \\ hu^2 + \frac{1}{2} gh^2 \end{bmatrix} = \begin{bmatrix} q^2 \\ \frac{(q^2)^2)}{q^1} + \frac{1}{2} g (q^1)^2 \end{bmatrix}.
$$
Note at this point that the flux is in-fact convex.

The quasi-linear form of the equations is defined by
$$
    q_t + f'(q) q_x = 0
$$
where
$$
    f'(q) = \begin{bmatrix}
        0 & 1 \\
        -(q^2 / q^1)^2 + g q^1 & 2 q^2 / q^1
    \end{bmatrix} = \begin{bmatrix}
        0 & 1 \\
        -u^2 + gh & 2u
    \end{bmatrix}.
$$
The eigensystem therfore has the structure
$$
    \lambda = u \pm \sqrt{gh} \text{ and } R = \begin{bmatrix}
        1 & 1 \\ u - \sqrt{gh} & u + \sqrt{gh} \end{bmatrix}
$$
These waves (speeds) are known as **gravity waves** (speeds) are are important for determining the behavior even in the simple case of a Riemann problem.  The non-dimensional parameter that is often used in these cases is often called the **Froude number** defined as}
$$
    \text{Fr} = \frac{|u|}{\sqrt{gh}}
$$
and is similar to the role of the speed of sound in gasses.

## Dam-Break and Riemann Problems